
## **Notebook Overview — Fixed-Effects Decomposition and Bootstrap**

 **Purpose:**
 This notebook estimates a fixed-effects (FE) model to decompose SES-related differences in essay scores  
 into *content*, *style*, and *other* components, and optionally computes bootstrap confidence intervals.

**How to Use:**
 - **Run up to Cell 3** if you only need the **decomposition dataset**:  
   This will fit the FE model once, append the fixed effects (`fe_essay`, `fe_k`) and residuals (`u`)  
   to each observation, and save the full augmented dataset as a CSV file.  
   --> Output file: `decomp_rows_with_fe.csv`

 - **Run all cells (1–7)** if you also want to **compute bootstrap confidence intervals**:  
   This will perform 500 clustered bootstrap replications at the essay level (resampling essays  
   with all rewrites), re-estimate the FE model each time, and produce percentile-based 95% CIs  
   for the total, content, style, and others decomposition metrics.

 **Outputs:**
 - `decomp_rows_with_fe.csv` --> Row-level data with FE components for all essays and rewrites.  
 - `summary_df` --> Compact table with decomposition point estimates and bootstrap 95% CIs (Both in .csv and .tex formats).

### **Cell 1 — Imports & Configuration**

**Purpose:**
- Import all core Python libraries used throughout the notebook.  
- Prepare tools for data handling (`pandas`, `numpy`), model estimation (`pyfixest`), and progress visualization (`tqdm`).  
- Include utilities for parallel computation (`joblib`) and statistical modeling (`statsmodels`).  
- Initialize a global random number generator (`rng`) with a fixed seed to ensure reproducibility of bootstrap samples.


In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from joblib import Parallel, delayed
import statsmodels.api as sm
from pyfixest.estimation import feols

# Reproducibility
rng = np.random.default_rng(42)

In [10]:
out_path = "../data/results/sat/"

table_out_path = "../tables/sat/tables/"

df_path = "../data/results/sat/data_sat_scored.csv"

### **Cell 2 — Load & Basic Preparation**

**Purpose:**
- Load the scored dataset containing essay-level and rewrite-level information.  
- Sort rows by `essay_id` and `k` so each essay’s rewrites appear sequentially, ensuring consistent ordering for later fixed-effect estimation and bootstrapping.  
- Reset the index to maintain a clean 0–N row index after sorting.


In [3]:
df = pd.read_csv(df_path, encoding="ISO-8859-1", index_col=0)

df = df.sort_values(["essay_id","k"]).reset_index(drop=True)

### **Cell 3 — Fit Fixed-Effects Model and Extract Baseline Estimates**

**Purpose:**
- Estimate a two-way fixed-effects model using `pyfixest` to control for essay-specific and prompt-specific variation.  
- Model specification: `score_high_full ~ 1 | essay_id + k`.  
- Retrieve the estimated fixed effects from the model output (`fes`):  
  - `fe_essay` → captures essay-level (content) differences.  
  - `fe_k` → captures prompt-level (context) effects.  
- Map these fixed effects back to each observation in the DataFrame for later decomposition.  
- Compute residuals `u = score_high_full – fe_essay`, representing the style component once essay-level effects are removed.


In [4]:
# FE regression
res = feols("score_high_full ~ 1 | essay_id + k", data=df)
print(res.summary)

fes = res.fixef()
fe_essay_map = fes["C(essay_id)"]  #
fe_k_map = fes["C(k)"]

# Map back to rows
df["fe_essay"] = df["essay_id"].map(fe_essay_map)
df["fe_k"] = df["k"].map(fe_k_map)

# Style residual (after removing essay FE only, as in your code)
df["u"] = df["score_high_full"] - df["fe_essay"]

# Compute diff column
df["diff"] = df["score_high_full"] - df["score_low_full"]

# Save DataFrame with FEs and residuals
df.to_csv(out_path + "decomp_rows_with_fe.csv", index=False)
print(f"Saved augmented dataset - {out_path}decomp_rows_with_fe.csv")

functools.partial(<function summary at 0x12ef547c0>, models=[<pyfixest.estimation.feols_.Feols object at 0x158b9a540>])
Saved augmented dataset - ../data/results/sat/decomp_rows_with_fe.csv


In [5]:
# Define a small utility function to compute conditional means while safely handling missing values (`NaN`). 

def mean_mask(s: pd.Series, mask: pd.Series) -> float:
    s2 = s[mask & s.notna()]
    return float(s2.mean()) if len(s2) else np.nan


### **Cell 4 — Decomposition**

**Purpose:**
- Compute the SES gap and its decomposition into **content**, **style**, and **others** using the FE outputs.  
- Restrict the decomposition to **k == 0** for comparability.  
- Return both **absolute gaps** and **shares** (each component divided by the total gap).

**Steps:**
 - **A) Total gap (k==0):**  
   - Group 0 = high-SES `score_high_full` where `low_SES==0`.  
   - Group 1 = low-SES  `score_low_full`  where `low_SES==1`.  
   - `total_gap = mean(group 0) – mean(group 1)`.
 - **B) Content gap (k==0):**  
   - Compare `fe_essay` means between SES groups; essay FE proxies content.  
   - `content_gap = mean(fe_essay | high-SES) – mean(fe_essay | low-SES)`.
 - **C) Style gap (k==0):**  
   - Use residual `u = score_high_full – fe_essay` as a style proxy.  
   - `style_gap = mean(u | high-SES) – mean(u | low-SES)`.
 - **D) Others gap (k==0, within high-SES):**  
   - Difference between `score_high_full` and `score_low_full` (same SES group).  
   - `others_gap = mean(score_high_full) – mean(score_low_full)` for `low_SES==0`.

**Outputs:**
 - Dictionary with absolute gaps: `total_gap`, `content_gap`, `style_gap`, `others_gap`.  
 - Shares: `share_content`, `share_style`, `share_others` (component / total_gap).  
 - `point_est` stores the baseline (non-bootstrap) decomposition for the full sample.

In [6]:
def compute_decomposition_point_estimates(df_in: pd.DataFrame):
    df_ = df_in.copy()

    # --- A) total gap (k==0)
    mA = (df_["low_SES"]==0) & (df_["k"]==0) & df_["score_high_full"].notna()
    mB = (df_["low_SES"]==1) & (df_["k"]==0) & df_["score_low_full"].notna()
    tmp_vals = pd.Series(np.nan, index=df_.index)
    tmp_vals[mA] = df_.loc[mA, "score_high_full"].astype(float)
    tmp_vals[mB] = df_.loc[mB, "score_low_full"].astype(float)

    group = pd.Series(np.nan, index=df_.index)
    group[mA] = 0
    group[mB] = 1

    m0 = (group==0)
    m1 = (group==1)

    mean0 = mean_mask(tmp_vals, m0)
    mean1 = mean_mask(tmp_vals, m1)
    total_gap = mean0 - mean1

    # --- B) content gap (k==0), use essay FE
    mK0 = (df_["k"]==0)
    content_mean0 = mean_mask(df_["fe_essay"], mK0 & (df_["low_SES"]==0))
    content_mean1 = mean_mask(df_["fe_essay"], mK0 & (df_["low_SES"]==1))
    content_gap = content_mean0 - content_mean1

    # --- C) style gap via residual u = score_high_full - fe_essay
    u_mean0 = mean_mask(df_["u"], mK0 & (df_["low_SES"]==0))
    u_mean1 = mean_mask(df_["u"], mK0 & (df_["low_SES"]==1))
    style_gap = u_mean0 - u_mean1

    # --- D) others gap (your original "diff" within high-SES at k==0)
    m_last = (df_["low_SES"]==0) & (df_["k"]==0)
    sh_mean = mean_mask(df_["score_high_full"], m_last)
    sl_mean = mean_mask(df_["score_low_full"],  m_last)
    others_gap = sh_mean - sl_mean

    return {
        "total_gap": total_gap,
        "content_gap": content_gap,
        "style_gap": style_gap,
        "others_gap": others_gap,
        "share_content": content_gap/total_gap if pd.notna(total_gap) and total_gap!=0 else np.nan,
        "share_style": style_gap/total_gap   if pd.notna(total_gap) and total_gap!=0 else np.nan,
        "share_others": others_gap/total_gap  if pd.notna(total_gap) and total_gap!=0 else np.nan,
    }

point_est = compute_decomposition_point_estimates(df)
point_est


{'total_gap': 0.6850525775383809,
 'content_gap': 0.49835744645038815,
 'style_gap': 0.1590474738030871,
 'others_gap': 0.07306129673577555,
 'share_content': 0.7274732813080571,
 'share_style': 0.23216827294424167,
 'share_others': 0.1066506413249459}

In [7]:
df.to_csv(out_path + "decomp_rows_with_fe.csv", index=False)


### **Cell 5 — Bootstrap (Clustered at essay_id; Parallelized, 500 Reps, 2.5–97.5% CI)**

**Purpose:**
- Estimate uncertainty for the decomposition metrics using **clustered bootstrapping** at the essay level.  
- Each bootstrap sample resamples essays (clusters) **with replacement**, keeping all their rewrites together.  
- The process re-fits the fixed-effects model on each resample, computes the decomposition again,  
  and aggregates results across 500 iterations to form **95% confidence intervals**.

**Steps:**
1. Reset DataFrame index and precompute a dictionary mapping each `essay_id` to its row indices  
2. Define the helper function `one_rep(seed)` that performs a **single bootstrap iteration**:  
   - Samples `essay_id`s with replacement (cluster bootstrap).  
   - Extracts the corresponding rows efficiently via `.iloc`.  
   - Re-fits the FE model (`score_high_full ~ 1 | essay_id + k`).  
   - Computes essay and prompt fixed effects, residuals, and decomposition metrics.  
3. Generate a list of independent random seeds (one per iteration) to ensure reproducibility  
   across parallel workers.  
4. Use **Joblib’s `Parallel`** and **`delayed`** utilities to run all 500 bootstrap replications concurrently  
   (`n_jobs=-1` automatically uses all available CPU cores).  
5. Convert the resulting list of dictionaries into a single DataFrame (`boot_df_stats`),  
   where each row corresponds to one bootstrap iteration and each column to a decomposition metric.  
6. Compute percentile-based confidence intervals (2.5% and 97.5%) for every statistic,  
   storing them in `ci_bounds` as a compact dictionary of lower–upper tuples.


In [8]:
B = 500  # number of bootstrap iterations


def run_bootstrap_for_group(df_group: pd.DataFrame, group_label: str, B: int = 500, seed: int = 42):
    """
    Runs the same bootstrap you already have, but restricted to df_group.
    Returns a list of rows (dicts) for this group, ready to concat into summary_df.
    """

    # --- Rebuild essay clusters inside the group ---
    df_g = df_group.reset_index(drop=True)
    essay_to_idx_g = df_g.groupby("essay_id").indices
    essay_ids_g = np.array(list(essay_to_idx_g.keys()))
    n_clusters_g = len(essay_ids_g)

    # Edge case: if group has too few essays, skip
    if n_clusters_g < 2:
        print(f"Skipping group {group_label}: only {n_clusters_g} distinct essays.")
        return []

    rng = np.random.default_rng(seed)
    seeds = rng.integers(0, 2**32 - 1, size=B, dtype=np.uint64)

    def one_rep_group(s: int) -> dict:
        """
        One bootstrap iteration for THIS group only.
        Same logic as your existing one_rep, but using df_g and essay_to_idx_g.
        """
        rng_local = np.random.default_rng(s)

        sampled = rng_local.choice(essay_ids_g, size=n_clusters_g, replace=True)
        idx = np.concatenate([essay_to_idx_g[eid] for eid in sampled])

        df_b = df_g.iloc[idx].copy()

        res_b = feols("score_high_full ~ 1 | essay_id + k", data=df_b)

        fes_b = res_b.fixef()
        fe_essay_b = fes_b["C(essay_id)"]
        fe_k_b = fes_b["C(k)"]

        df_b["fe_essay"] = df_b["essay_id"].map(fe_essay_b)
        df_b["fe_k"] = df_b["k"].map(fe_k_b)
        df_b["u"] = df_b["score_high_full"] - df_b["fe_essay"]

        return compute_decomposition_point_estimates(df_b)

    print(f"Running {B} bootstrap iterations for group: {group_label} (N={len(df_g)}, essays={n_clusters_g})")

    results = Parallel(n_jobs=-1, backend="loky")(
        delayed(one_rep_group)(int(s)) for s in tqdm(seeds)
    )

    boot_df_stats = pd.DataFrame(results)

    # --- Compute CIs ---
    ci_bounds = {}
    for col in boot_df_stats.columns:
        low, high = np.nanpercentile(boot_df_stats[col], [2.5, 97.5])
        ci_bounds[col] = (low, high)

    # --- Point estimates on the ORIGINAL (non-resampled) group ---
    point_est = compute_decomposition_point_estimates(df_g)

    # --- Build rows for this group ---
    rows = []
    for stat_name, pe in point_est.items():
        lo, hi = ci_bounds[stat_name]
        rows.append({
            "group": group_label,
            "stat": stat_name,
            "point_estimate": round(pe, 3),
            "ci_2.5": round(lo, 3),
            "ci_97.5": round(hi, 3),
        })

    return rows


In [9]:
all_rows = []

# Overall
all_rows += run_bootstrap_for_group(df, "Overall", B=B, seed=42)

# White / Non-White
all_rows += run_bootstrap_for_group(df[df["race_white"] == True],  "White",      B=B, seed=101)
all_rows += run_bootstrap_for_group(df[df["race_white"] == False], "Non-White",  B=B, seed=102)

# Male / Female
all_rows += run_bootstrap_for_group(df[df["gender_male"] == True],  "Male",   B=B, seed=201)
all_rows += run_bootstrap_for_group(df[df["gender_male"] == False], "Female", B=B, seed=202)

# All different prompts in prompt_name
for pname in sorted(df["prompt_name"].dropna().unique()):
    mask = df["prompt_name"] == pname
    label = f"Prompt={pname}"
    all_rows += run_bootstrap_for_group(df[mask], label, B=B, seed=300 + hash(pname) % 1000)

# All different grades in grade_level
for g in sorted(df["grade_level"].dropna().unique()):
    mask = df["grade_level"] == g
    label = f"Grade={g}"
    all_rows += run_bootstrap_for_group(df[mask], label, B=B, seed=400 + int(g))

# Final combined table
summary_df = pd.DataFrame(all_rows)
summary_df


Running 500 bootstrap iterations for group: Overall (N=6706, essays=958)


100%|██████████| 500/500 [00:15<00:00, 31.59it/s]


Running 500 bootstrap iterations for group: White (N=2870, essays=410)


100%|██████████| 500/500 [00:06<00:00, 71.92it/s]


Running 500 bootstrap iterations for group: Non-White (N=3836, essays=548)


...

Running 500 bootstrap iterations for group: Grade=11.0 (N=917, essays=131)


100%|██████████| 500/500 [00:05<00:00, 92.22it/s] 


Running 500 bootstrap iterations for group: Grade=12.0 (N=161, essays=23)


100%|██████████| 500/500 [00:05<00:00, 98.98it/s] 


,group,stat,point_estimate,ci_2.5,ci_97.5
0,Overall,total_gap,0.685,0.575,0.798
1,Overall,content_gap,0.498,0.408,0.600
2,Overall,style_gap,0.159,0.119,0.197
3,Overall,others_gap,0.073,0.053,0.093
4,Overall,share_content,0.727,0.683,0.774
...,...,...,...,...,...
149,Grade=12.0,style_gap,0.256,-0.076,0.578
150,Grade=12.0,others_gap,0.001,-0.114,0.108
151,Grade=12.0,share_content,2.302,-8.393,7.211
152,Grade=12.0,share_style,-1.995,-8.716,9.960


### **Cell 7 — Summary Table (Point Estimates + Bootstrap Confidence Intervals)**

**Purpose:**
- Combine the **baseline decomposition point estimates** (from Cell 5)  
with the **bootstrap confidence intervals** (from Cell 6) into one concise summary table.  

In [14]:
GROUP_LABELS = {
    "All": "All",
    "race_white=True": "White",
    "race_white=False": "Non-White",
    "gender_male=True": "Male",
    "gender_male=False": "Female",
    "Grade=6.0": "Grade 6",
    "Grade=8.0": "Grade 8",
    "Grade=9.0": "Grade 9",
    "Grade=10.0": "Grade 10",
    "Grade=11.0": "Grade 11",
    "Grade=12.0": "Grade 12",
}

# Format estimate + CI into a single compact value
def format_ci(row):
    return f"{row['point_estimate']:.3f} [{row['ci_2.5']:.3f}, {row['ci_97.5']:.3f}]"

summary_df_display = summary_df.copy()

summary_df_display["estimate_ci"] = summary_df_display.apply(format_ci, axis=1)

# Map raw group names to labels
summary_df_display["group"] = summary_df_display["group"].map(
    lambda x: GROUP_LABELS.get(x, x)  # fallback = original (e.g., prompts)
)

visual_table = (
    summary_df_display
    .pivot(index="stat", columns="group", values="estimate_ci")
    .sort_index(axis=1)
)

# -> First: group labels in the exact order defined
ordered_group_names = list(GROUP_LABELS.values())

# -> Then: prompt groups (anything starting with "Prompt")
prompt_cols = sorted([col for col in visual_table.columns if str(col).startswith("Prompt")])

# -> Then: any remaining unexpected groups
other_cols = [
    col for col in visual_table.columns
    if col not in ordered_group_names and col not in prompt_cols
]

# Final column order
final_order = ordered_group_names + prompt_cols + other_cols
final_order = [col for col in final_order if col in visual_table.columns]

visual_table = visual_table[final_order]


visual_table = (
    visual_table
    .rename_axis(index=None, columns=None)  # drop 'stat' and 'group' axis names
)

visual_table


,White,Non-White,Male,Female,Grade 6,Grade 8,Grade 10,Grade 11,Grade 12,"Prompt=""A Cowboy Who Rode the Waves""",...,Prompt=Distance learning,Prompt=Driverless cars,Prompt=Exploring Venus,Prompt=Facial action coding system,Prompt=Grades for extracurricular activities,Prompt=Mandatory extracurricular activities,Prompt=Seeking multiple opinions,Prompt=Summer projects,Prompt=The Face on Mars,Overall
content_gap,"0.405 [0.234, 0.572]","0.652 [0.492, 0.799]","0.494 [0.347, 0.659]","0.504 [0.360, 0.658]","0.315 [0.045, 0.570]","0.479 [0.338, 0.614]","0.367 [0.186, 0.555]","0.248 [0.055, 0.454]","-0.296 [-0.987, 0.314]","0.315 [0.027, 0.575]",...,"0.198 [-0.063, 0.441]","0.283 [0.018, 0.578]","0.407 [0.010, 0.806]","0.448 [0.131, 0.779]","0.458 [0.207, 0.693]","0.635 [0.359, 0.942]","0.216 [-0.120, 0.539]","0.376 [0.109, 0.621]","0.369 [-0.011, 0.652]","0.498 [0.408, 0.600]"
others_gap,"0.084 [0.054, 0.117]","0.057 [0.030, 0.086]","0.057 [0.028, 0.085]","0.088 [0.060, 0.118]","0.095 [0.005, 0.181]","0.056 [0.030, 0.082]","0.077 [0.025, 0.130]","0.111 [0.071, 0.153]","0.001 [-0.114, 0.108]","0.095 [0.005, 0.182]",...,"0.030 [-0.013, 0.074]","-0.018 [-0.072, 0.038]","0.251 [0.107, 0.418]","0.099 [0.025, 0.159]","0.051 [0.004, 0.106]","0.079 [0.016, 0.151]","0.060 [0.006, 0.108]","0.152 [0.095, 0.214]","-0.009 [-0.079, 0.058]","0.073 [0.053, 0.093]"
share_content,"0.743 [0.616, 0.834]","0.751 [0.693, 0.805]","0.721 [0.646, 0.790]","0.734 [0.664, 0.801]","0.741 [0.299, 1.054]","0.748 [0.677, 0.813]","0.688 [0.517, 0.810]","0.556 [0.264, 0.719]","2.302 [-8.393, 7.211]","0.741 [0.303, 1.066]",...,"0.380 [-0.319, 0.581]","1.033 [-0.146, 1.865]","0.599 [0.042, 0.757]","0.738 [0.462, 0.949]","0.797 [0.599, 0.973]","0.741 [0.568, 0.879]","0.512 [-0.742, 0.825]","0.576 [0.317, 0.708]","0.708 [0.070, 0.932]","0.727 [0.683, 0.774]"
share_others,"0.154 [0.097, 0.252]","0.066 [0.037, 0.100]","0.082 [0.040, 0.128]","0.129 [0.085, 0.187]","0.223 [0.001, 0.656]","0.087 [0.048, 0.142]","0.144 [0.052, 0.273]","0.249 [0.141, 0.597]","-0.010 [-1.035, 1.684]","0.223 [0.015, 0.602]",...,"0.058 [-0.033, 0.201]","-0.065 [-0.487, 0.602]","0.369 [0.167, 0.987]","0.163 [0.053, 0.317]","0.089 [0.008, 0.222]","0.092 [0.020, 0.207]","0.141 [0.004, 0.824]","0.232 [0.135, 0.460]","-0.017 [-0.328, 0.122]","0.107 [0.078, 0.140]"
share_style,"0.231 [0.146, 0.349]","0.212 [0.161, 0.267]","0.253 [0.188, 0.326]","0.214 [0.146, 0.275]","0.215 [-0.128, 0.706]","0.205 [0.145, 0.287]","0.254 [0.126, 0.405]","0.321 [0.097, 0.564]","-1.995 [-8.716, 9.960]","0.215 [-0.113, 0.636]",...,"0.648 [0.438, 1.380]","0.229 [-0.454, 1.839]","0.116 [-0.182, 0.318]","0.232 [0.004, 0.509]","0.202 [0.023, 0.386]","0.179 [0.045, 0.325]","0.275 [-0.032, 1.085]","0.385 [0.230, 0.574]","0.034 [-0.568, 0.232]","0.232 [0.186, 0.280]"
style_gap,"0.126 [0.072, 0.184]","0.184 [0.131, 0.243]","0.173 [0.120, 0.223]","0.147 [0.091, 0.202]","0.091 [-0.041, 0.206]","0.131 [0.086, 0.178]","0.136 [0.064, 0.210]","0.143 [0.026, 0.257]","0.256 [-0.076, 0.578]","0.091 [-0.028, 0.214]",...,"0.337 [0.168, 0.503]","0.063 [-0.049, 0.190]","0.079 [-0.055, 0.233]","0.141 [0.001, 0.263]","0.116 [0.010, 0.230]","0.153 [0.032, 0.274]","0.116 [0.001, 0.228]","0.252 [0.098, 0.398]","0.018 [-0.101, 0.120]","0.159 [0.119, 0.197]"
total_gap,"0.545 [0.362, 0.741]","0.869 [0.678, 1.051]","0.686 [0.521, 0.862]","0.687 [0.523, 0.852]","0.425 [0.120, 0.701]","0.640 [0.490, 0.791]","0.534 [0.319, 0.743]","0.447 [0.190, 0.720]","-0.129 [-0.745, 0.526]","0.425 [0.118, 0.726]",...,"0.521 [0.172, 0.838]","0.274 [-0.004, 0.588]","0.681 [0.204, 1.152]","0.607 [0.217, 0.980]","0.574 [0.291, 0.861]","0.857 [0.572, 1.186]","0.422 [0.057, 0.780]","0.654 [0.312, 0.981]","0.521 [0.089, 0.871]","0.685 [0.575, 0.798]"


In [ ]:
visual_table.to_csv(
    table_out_path + "decomposition_bootstrap_summary_visual.csv"
)

visual_table.to_latex(
    table_out_path + "decomposition_bootstrap_summary_visual.tex",
    escape=False
)